# Code Generation Demo 

## Description

This notebook demonstrates how to create a simple code generation interface using CodeLlama models from Hugging Face's `transformers` library and `gradio` for the user interface.

### Main Features:
- Select from a list of pre-trained CodeLlama models.
- Input a description of the code you want, and receive generated code based on the prompt.
- Cache models to avoid reloading them multiple times.


## Log in to Hugging Face Hub

In this cell, we import the `login` function from the Hugging Face Hub and call it to authenticate with your Hugging Face account. This step is required to access private models, datasets, and other resources hosted on Hugging Face.

### Directions to Generate Access Token:
1. Go to the [Hugging Face website](https://huggingface.co/).
2. Log in to your Hugging Face account.
3. Navigate to your **profile icon** on the top right, and click **Settings**.
4. Under **Access Tokens** (on the left sidebar), click **New Token** to generate a new access token.
5. Copy the generated token.

### Usage:
When you run this cell, you'll be prompted to paste the access token, which grants access to your Hugging Face resources.

> **Do not share your Access Tokens with anyone**

In [ ]:
from huggingface_hub import login
login()

## Import Required Libraries

In [ ]:
import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Basic Usage

Below there are a list of available CodeLlama models to choose from. The function, `load_model`, loads the selected CodeLlama model along with its tokenizer. It uses Hugging Face's `AutoModelForCausalLM` and `AutoTokenizer` to load the pre-trained model and then returns a tuple containing the loaded model and tokenizer. Note that, you need to request access for CodeLlama models to use them. 

In [ ]:
code_llama_models = {
    "CodeLlama 7B Instruct": "meta-llama/CodeLlama-7b-Instruct-hf",
    "CodeLlama 13B Instruct": "meta-llama/CodeLlama-13b-Instruct-hf",
    "CodeLlama 34B Instruct": "meta-llama/CodeLlama-34b-Instruct-hf",
    "CodeLlama 70B Instruct": "meta-llama/CodeLlama-70b-Instruct-hf",
}

In [ ]:
def load_model(model_name):
    """Load the specified CodeLlama model."""
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto')
    model.to(device)
    return model, tokenizer

Cache models to avoid reloading

In [ ]:
model_cache = {}

In [ ]:
def generate_code(prompt, model_choice):
    """Generate code using the selected CodeLlama model."""
    if model_choice not in model_cache:
        model_cache[model_choice] = load_model(code_llama_models[model_choice])
    model, tokenizer = model_cache[model_choice]
    full_prompt = f"Write a program based on the following description:\n\n\"{prompt}\"\n\nCode:\n"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=512,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
    )
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    code = generated_code.split("Code:")[-1].strip()
    return code

The cell below defines a simple Gradio interface. The interface consists of a dropdown menu for users to select specific CodeLlama model, a code output block is used to display the generated code, specifically formatted for Python, a textbox to enter queries and a window that displays the conversation between the user and the assistant.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("<h1><center>Code Generation with CodeLlama Models</center></h1>")
    model_choice = gr.Dropdown(list(code_llama_models.keys()), label="Select CodeLlama Model")
    prompt_input = gr.Textbox(label="Describe the code you want", lines=2)
    code_output = gr.Code(language="python", label="Generated Code")
    generate_button = gr.Button("Generate Code")
    generate_button.click(generate_code, [prompt_input, model_choice], code_output)

Launch the interface

In [ ]:
demo.launch()